In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchsummary import summary

In [13]:
# 열 이름
column_names = ["A", "B", "C", "D", "E", "F", "G", "target"]

# 파싱 함수
def parse_line_to_list(line):
    line = tf.strings.strip(line)
    parts = tf.strings.split(line, sep="\t")
    parts = tf.strings.to_number(parts, out_type=tf.float32)
    return parts

# 유효한 줄만 통과
def is_valid_line(line):
    line = tf.strings.strip(line)
    parts = tf.strings.split(line, sep="\t")
    return tf.equal(tf.size(parts), 8)

# Dataset
dataset = tf.data.TextLineDataset("C:/Users/하태권/Downloads/seeds/seeds_dataset.txt") \
            .filter(is_valid_line) \
            .map(parse_line_to_list)

# 리스트로 변환
data_list = [list(row.numpy()) for row in dataset]

# DataFrame으로 변환
df = pd.DataFrame(data_list, columns=column_names)

df

,A,B,C,D,E,F,G,target
0,15.260000,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.880000,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.290000,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.840000,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.139999,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
...,...,...,...,...,...,...,...,...
194,12.190000,13.20,0.8783,5.137,2.981,3.631,4.870,3.0
195,11.230000,12.88,0.8511,5.140,2.795,4.325,5.003,3.0
196,13.200000,13.66,0.8883,5.236,3.232,8.315,5.056,3.0
197,11.840000,13.21,0.8521,5.175,2.836,3.598,5.044,3.0


In [15]:
df.isnull().sum()

A         0
B         0
C         0
D         0
E         0
F         0
G         0
target    0
dtype: int64

In [17]:
X = df.drop('target',axis=1)
X.head()

,A,B,C,D,E,F,G
0,15.260000,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.880000,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.290000,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.840000,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.139999,14.99,0.9034,5.658,3.562,1.355,5.175


In [19]:
y = df["target"].astype(int) - 1

print(y.value_counts())

target
1    68
0    66
2    65
Name: count, dtype: int64


In [21]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, shuffle=True, random_state=12)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(159, 7) (159,)
(40, 7) (40,)


In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(7,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

C:\Userai\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,443 (29.07 KB)

 Trainable params: 7,443 (29.07 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile( optimizer=Adam(),loss=SparseCategoricalCrossentropy(),metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.1)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8784 - loss: 0.5854 - val_accuracy: 0.8125 - val_loss: 0.5206
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8799 - loss: 0.5860 - val_accuracy: 0.8125 - val_loss: 0.5170
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8668 - loss: 0.5689 - val_accuracy: 0.8125 - val_loss: 0.5158
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9083 - loss: 0.5220 - val_accuracy: 0.9375 - val_loss: 0.5090
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8840 - loss: 0.5246 - val_accuracy: 0.9375 - val_loss: 0.4729
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8932 - loss: 0.5173 - val_accuracy: 0.9375 - val_loss: 0.4418
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8679 - loss: 0.5135 - val_accuracy: 0.9375 - val_loss: 0.4579
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8990 - loss: 0.4972 - val_accuracy: 0.8750 - v

In [31]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(7,))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='sigmoid')(x)
prediction = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=prediction)

model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,443 (29.07 KB)

 Trainable params: 7,443 (29.07 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
history = model.fit(X_train, y_train,epochs=20,batch_size=16, validation_data=(X_val, y_val))

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.2593 - loss: 1.1240 - val_accuracy: 0.3500 - val_loss: 1.0925
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3499 - loss: 1.0865 - val_accuracy: 0.3750 - val_loss: 1.0794
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3717 - loss: 1.0752 - val_accuracy: 0.4500 - val_loss: 1.0497
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5096 - loss: 1.0565 - val_accuracy: 0.5750 - val_loss: 1.0207
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5551 - loss: 1.0130 - val_accuracy: 0.6500 - val_loss: 0.9867
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6396 - loss: 0.9801 - val_accuracy: 0.6000 - val_loss: 0.9637
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5922 - loss: 0.9522 - val_accuracy: 0.6750 - val_loss: 0.9295
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6404 - loss: 0.9293 - val_accuracy: 0.6750 - v

In [35]:
class SimpleMLP(Model):
    
    def __init__(self, num_classes):
        super(SimpleMLP, self).__init__(name='mlp')
        self.num_classes = num_classes
        
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.dense3 = Dense(32, activation='relu')
        self.dense4 = Dense(16, activation='relu')
        self.dense5 = Dense(8, activation='sigmoid')
        self.dense6 = Dense(num_classes, activation='softmax')  
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return self.dense6(x)

In [37]:
model = SimpleMLP(num_classes=3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.2769 - loss: 1.3392 - val_accuracy: 0.0312 - val_loss: 1.1414
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1937 - loss: 1.2025 - val_accuracy: 0.3438 - val_loss: 1.1266
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3191 - loss: 1.1958 - val_accuracy: 0.3438 - val_loss: 1.0941
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3776 - loss: 1.1148 - val_accuracy: 0.3438 - val_loss: 1.0812
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3459 - loss: 1.1081 - val_accuracy: 0.3438 - val_loss: 1.0656
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3090 - loss: 1.1108 - val_accuracy: 0.5312 - val_loss: 1.0517
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3661 - loss: 1.0809 - val_accuracy: 0.3438 - val_loss: 1.0422
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3952 - loss: 1.0316 - val_accuracy: 0.4375 - val_loss: 1.0195
